## Ejercicio 1: Unidades de capacidad y particionamiento de DynamoDB
En este ejercicio, se cargan datos en tablas de DynamoDB aprovisionadas con diferentes unidades de capacidad de escritura/lectura y se comparan los tiempos de carga de diferentes conjuntos de datos. En primer lugar, cargue un conjunto de datos más pequeño en una tabla y observe los rápidos tiempos de ejecución. A continuación, cargue un conjunto de datos más grande en una tabla infraaprovisionada para simular excepciones de estrangulamiento. Por último, se simula la contrapresión del índice secundario global en una tabla creando una tabla con mayor aprovisionamiento y un índice secundario global con sólo 1 unidad de capacidad de escritura (WCU). En este ejercicio, se utilizan datos de registro de acceso al servidor web de muestra, similares a los datos de registro del servidor web generados por Apache.

In [ ]:
import boto3
from botocore.exceptions import ClientError
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from spdynamodb import DynamoTable
from datetime import datetime
import json
import time
from decimal import Decimal

In [ ]:
dt = DynamoTable()
try:
    dt.select_table('LogFileTable')
    print(dt)
except:
    dt.create_table(
        table_name='LogFileTable',
        partition_key='PK',
        partition_key_type='S',
        read_capacity=5,
        write_capacity=5,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

In [ ]:
dt.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    index_name="GSI1",
    proj_type=["bytessent"],
    read_capacity=5,
    write_capacity=5
)

In [13]:
status = dt.check_status_gsi()
start = time.time()
if status == 'CREATING':
    print("Global secondary index is being created, this may take a few minutes...")
    try:
        while status == 'CREATING':
            status = dt.check_status_gsi()
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    else:  
        end = time.time()
        minute = (end - start) / 60
        print("Global secondary index created. Time elapsed: {0:.2f} minute".format(minute))

Global secondary index is being created, this may take a few minutes...


Los atributos especiales incluyen atributos con nombre que definen la clave principal de una tabla de DynamoDB o un índice secundario global (GSI). Los índices secundarios globales tienen claves primarias al igual que las tablas de DynamoDB. En DynamoDB, la clave de partición es la misma que la clave hash, y la clave de ordenación es la misma que la clave de rango. Las API de DynamoDB utilizan los términos hash y rango, mientras que la documentación de AWS utiliza los términos partición y rango. Independientemente de los términos que utilice, estas dos claves juntas forman la clave principal.

In [ ]:
#columns = ['requestid','host','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
columns = ['PK','GSI1-PK','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
df_logs = pd.read_csv('data/logfile_small1.csv', names=columns, header=None)
df_logs['PK'] = df_logs['PK'].astype(str)

In [ ]:
df_logs.head()

In [ ]:
len(df_logs)

In [ ]:
dt.batch_pandas(df_logs)

In [ ]:
columns = ['PK','GSI1-PK','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
df_logs = pd.read_csv('data/logfile_medium1.csv', names=columns, header=None)
df_logs['PK'] = df_logs['PK'].astype(str)

In [ ]:
df_logs.head()

In [ ]:
len(df_logs)

In [ ]:
dt.batch_pandas(df_logs)

### Increase the capacity of the table

In [ ]:
dt.table.meta.client.update_table(
    TableName=dt.table_name,
    ProvisionedThroughput={
        'ReadCapacityUnits': 105,
        'WriteCapacityUnits': 105
    }
)

waiter = dt.table.meta.client.get_waiter('table_exists')
waiter.wait(
    TableName=dt.table_name,
    WaiterConfig={
        'MaxAttempts': 10
    }
)

In [ ]:
columns = ['PK','GSI1-PK','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
df_logs = pd.read_csv('data/logfile_medium2.csv', names=columns, header=None)
df_logs['PK'] = df_logs['PK'].astype(str)

In [ ]:
df_logs.head()

In [ ]:
len(df_logs)

In [ ]:
dt.batch_pandas(df_logs)

Con la nueva capacidad, el tiempo total de carga es menor.

### Crear una nueva tabla con un índice secundario global de baja capacidad

In [ ]:
dt_gsi_low = DynamoTable()
try:
    dt_gsi_low.select_table('logfile_gsi_low')
    print(dt)
except:
    dt_gsi_low.create_table(
        table_name='logfile_gsi_low',
        partition_key='PK',
        partition_key_type='S',
        read_capacity=200,
        write_capacity=200,
        tags=[{'Key': 'workshop-design-patterns', 'Value': 'targeted-for-cleanup'}]
)

In [ ]:
dt_gsi_low.create_global_secondary_index(
    att_name="GSI1-PK",
    att_type="S",
    index_name="GSI1",
    proj_type=["bytessent"],
    read_capacity=1,
    write_capacity=1
)

In [ ]:
status = dt_gsi_low.check_status_gsi()
while status == 'CREATING':
    status = dt_gsi_low.check_status_gsi()
    time.sleep(30)
print("Global secondary index created.")

In [ ]:
columns = ['PK','GSI1-PK','date','hourofday','timezone','method','url','responsecode','bytessent','useragent']
df_logs_1 = pd.read_csv('data/logfile_medium1.csv', names=columns, header=None)
df_logs_1['PK'] = df_logs_1['PK'].astype(str)
df_logs_2 = pd.read_csv('data/logfile_medium2.csv', names=columns, header=None)
df_logs_2['PK'] = df_logs_2['PK'].astype(str)
df_logs = pd.concat([df_logs_1, df_logs_2])

In [ ]:
len(df_logs)

In [ ]:
dt_gsi_low.batch_pandas(df_logs)

### ProvisionedThroughputExceededException

DynamoDB puede devolver una excepción denominada `ProvisionedThroughputExceededException` con un mensaje de excepción que sugiere que se aumente la capacidad aprovisionada del índice secundario global. Se trata de un error revelador sobre el que hay que actuar. En resumen, si desea que el 100% de las escrituras de la tabla base de DynamoDB se copien en el GSI, entonces el GSI debe aprovisionarse con el 100% (la misma cantidad) de la capacidad de la tabla base, que debería ser 1.000 WCU en este ejemplo. En pocas palabras, el índice secundario global estaba infraaprovisionado.

Si la aplicación supera la capacidad de rendimiento aprovisionada en una tabla o un índice, las solicitudes podrían ser objeto de una limitación controlada. La limitación controlada impide que la aplicación consuma demasiadas unidades de capacidad. Cuando DynamoDB aplica una limitación a una operación de lectura o escritura, devuelve una ProvisionedThroughputExceededException al iniciador. A continuación, la aplicación puede adoptar las medidas pertinentes, como esperar un breve intervalo de tiempo antes de repetir la solicitud.

Cuando los estrangulamientos de escritura de un índice secundario global de DynamoDB son suficientes para crear solicitudes estranguladas, el comportamiento se denomina contrapresión del índice secundario global. Las solicitudes estranguladas son errores ProvisionedThroughputExceededException en los SDK de AWS, generan métricas ThrottledRequests en CloudWatch y aparecen como «solicitudes de escritura estranguladas» en la tabla base de la consola de AWS. Cuando se produce la contrapresión de GSI, se rechazan todas las escrituras en la tabla de DynamoDB hasta que se abre espacio en el búfer entre la tabla base de DynamoDB y GSI. Independientemente de si una nueva fila está destinada a un GSI, las escrituras durante un tiempo se rechazarán en la tabla base hasta que haya espacio disponible - DynamoDB no tiene tiempo para determinar si una fila que se va a escribir estará en el GSI o no. Se trata de una situación problemática, pero es una restricción inevitable de DynamoDB porque el servicio no puede crear un búfer de tamaño ilimitado entre la tabla base y el GSI; debe haber un límite para el número de elementos que esperan ser copiados de la tabla base a un GSI. Para detectar este comportamiento con antelación, es importante supervisar las solicitudes y los eventos limitados en la tabla de DynamoDB y el índice secundario global.

Recuerde que una tabla de DynamoDB se aprovisiona por separado de un índice secundario global. Si el índice secundario global no se aprovisiona lo suficiente, es posible que empiece a ejercer presión sobre las tablas en forma de estrangulamientos. La contrapresión es problemática porque hará que se rechacen todas las solicitudes de escritura en la tabla base hasta que el búfer entre la tabla base y el índice secundario global tenga espacio suficiente para los nuevos datos. Recuerde supervisar las métricas de CloudWatch tanto en sus tablas como en los índices secundarios globales, y establezca alarmas de supervisión basadas en sus requisitos empresariales.